In [ ]:
reload_lamb()

### Existential closure

The simplest kind of operation to add is a unary operation, and existential closure is a good example of this.  (See the type shifting notebook for more examples of unary operations.)

One way to define a composition operation is to write a python function that performs the operation.  Many details of this function may not be immediately obvious as it needs to interact quite a bit with the `lamb.lang` API.

In [ ]:
def ec_fun_direct(f, assignment=None):
    ts = meta.get_type_system() # load the current type system
    if not (ts.eq_check(f.type, types.type_property)): # is the type a property type?  (<e,t>)
        raise types.TypeMismatch(f, None, "Existential closure for <e,t>")  # if not, raise an exception.
    ec_expr = te("lambda f_<e,t> : Exists x_e : f(x)") # see below for discussion of this.
    result = ec_expr.apply(f.content.under_assignment(assignment)).reduce_sub(1) # reduce the body as well in case f was itself a lambda expression.
    # this much brute force is not necessary:
    #result = ec_expr.apply(f.content).reduce_all()
    return lang.UnaryComposite(f, result, source="EC(%s)" % (f.name))

ec_fun_direct(lang.cat).content.derivation

Fortunately, there is often a much sumpler way to introduce a unary composition operation.  Notice that the above function relies on a functional version of existential closure defined as `ec_expr`: this is what is sometimes called a combinator (really just a function with no free variables).  All the extra stuff is a wrapping around this.  Even the type checking is present in the combinator definition, so programmatic type checking can be done automatically.  Here is the combinator again:

In [ ]:
ec_combinator = te("lambda f_<e,t> : Exists x_e : f(x)")
ec_combinator

Any metalanguage function is also a python function, so you can do stuff like the following directly:

In [ ]:
ec_combinator(lang.cat.content)

Finally, there is a factory function in the `lang` module that constructs a unary composition operation from a combinator.  This can be fed into the compositional system's add_rule function.

In [ ]:
ec_rule = lang.unary_factory(ec_combinator, "EC", typeshift=False)
ec_rule(lang.cat).content.derivation

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.UnaryCompositionOp("EC", ec_rule))
lang.set_system(system)
system

In [ ]:
lang.cat * lang.gray

In [ ]:
ectest = (lang.cat * lang.gray) * None
ectest

In [ ]:
ectest.tree(derivations=True)

In [ ]:
ectest.trace()

### Binding evaluation parameters

A second kind of unary operation involves abstracting over a free variable in the metalanguage expression.  This can be thought of as "monstrous" shifting in the sense of Kaplan.  The following code sketches an implementation of this.

In [ ]:
reload_lamb()

In [ ]:
system = lang.td_system.copy()
speaker = lang.te("speaker_e")
system.assign_controller = lang.AssignmentController(specials=[speaker])
lang.set_system(system)
# note that the predefined examples in lang are defined with a different composition system that has no controller

In [ ]:
i = lang.Item("I", lang.te("speaker_e"))
i

In [ ]:
def monster_fun(f, assignment=None):
    new_a = lang.Assignment(assignment)
    new_a.update({"speaker": lang.te("x_e")})
    result = meta.LFun(types.type_e, f.content.under_assignment(new_a), varname="x")
    return lang.UnaryComposite(f, result, source="M(%s)" % (f.name))

monster_fun(i)

In [ ]:
system.add_rule(lang.UnaryCompositionOp("Monster", monster_fun))

In [ ]:
m_test = i * None
m_test

In [ ]:
(lang.cat * i) * None

In [ ]:
lang.cat * None

In [ ]:
reload_lamb()

### Predicate Modification

What about binary composition operations?  Predicate modification comes built in, but it is useful to see how one might construct PM.  Like the built-in version, this is restricted to type `<e,t>`, but please see the neo-Davidsonian notebook for a generalized version of PM.

Once again, you could write a python function that does the work.
  * Tangent: your instinct may be to construct the result directly by building up the right TypedExpression.  This is certainly possible, but it is surprisingly tricky to get right; I encourage you to find solutions that involve combinators.

In [ ]:
pm_op = lang.te("L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)")

def pm_fun2(fun1, fun2, assignment=None):
    """H&K predicate modification -- restricted to type <e,t>."""
    ts = meta.get_type_system()
    if not (ts.eq_check(fun1.type, types.type_property) and 
            ts.eq_check(fun2.type, types.type_property)):
        raise TypeMismatch(fun1, fun2, "Predicate Modification")
    c1 = fun1.content.under_assignment(assignment)
    c2 = fun2.content.under_assignment(assignment)
    result = pm_op.apply(c1).apply(c2).reduce_all()
    return lang.BinaryComposite(fun1, fun2, result)

In [ ]:
pm_fun2(lang.cat, lang.gray).content.derivation

However, there is once again a combinator at the heart of this, and PM can be constructed directly using this combinator.

In [ ]:
pm_op

In [ ]:
system = lang.td_system.copy()
system.remove_rule("PM")
#system.add_rule(lang.BinaryCompositionOp("PM2", pm_fun2)) # this would use the function defined directly
system.add_rule(lang.binary_factory_curried(pm_op, "PM2"))
lang.set_system(system)
system

In [ ]:
(lang.cat * lang.gray).tree()